In [1]:
# In case iPython does not find our personalized modules and we want to import them manually
# import sys
# sys.path.append('my/path/to/module/folder')
# import module_of_interest

# We can also make sure what's the main directory iPhython consider for running
# import os
# os.getcwd()

import constants
import requests
import pandas as pd
import numpy as np
import psycopg2
import csv
import datetime

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)


In [2]:
website_frame = pd.read_csv(constants.CSV_SOURCE_URL)

In [3]:
website_frame.shape

(380, 106)

In [4]:
# Remove unwanted columns

unwanted_cols = ["Div", "BWH", "BWD", "BWA", "IWH", "IWD", "IWA", "PSH", "PSD", "PSA", "WHH", "WHD", "WHA", "VCH", "VCD", "VCA",
                 "P>2.5", "P<2.5","AHh", "B365AHH", "B365AHA", "PAHH", "PAHA", "MaxAHH", "MaxAHA", "AvgAHH", "AvgAHA", "B365CH",
                 "B365CD", "B365CA", "BWCH", "BWCD", "BWCA", "IWCH", "IWCD", "IWCA", "PSCH", "PSCD", "PSCA", "WHCH", "WHCD", "WHCA",
                 "VCCH", "VCCD", "VCCA", "MaxCH", "MaxCD", "MaxCA", "AvgCH", "AvgCD", "AvgCA", "B365C>2.5", "B365C<2.5", "PC>2.5",
                 "PC<2.5", "MaxC>2.5", "MaxC<2.5", "AvgC>2.5", "AvgC<2.5", "AHCh", "B365CAHH", "B365CAHA", "PCAHH", "PCAHA", "MaxCAHH",
                 "MaxCAHA", "AvgCAHH", "AvgCAHA"]

website_frame.drop(columns = unwanted_cols, inplace = True)

In [5]:
# Rename columns
website_frame.rename(columns = {"FTHG": "FullTimeHomeTeamGoals",
                               "FTAG": "FullTimeAwayTeamGoals",
                               "FTR": "FullTimeResult",
                               "HTHG": "HalfTimeHomeTeamGoals",
                               "HTAG": "HalfTimeAwayTeamGoals",
                               "HTR": "HalfTimeResult",
                               "HS": "HomeTeamShots",
                               "AS": "AwayTeamShots",
                               "HST": "HomeTeamShotsOnTarget",
                               "AST": "AwayTeamShotsOnTarget",
                               "HF": "HomeTeamFouls",
                               "AF": "AwayTeamFouls",
                               "HC": "HomeTeamCorners",
                               "AC": "AwayTeamCorners",
                               "HY": "HomeTeamYellowCards",
                               "AY": "AwayTeamYellowCards",
                               "HR": "HomeTeamRedCards",
                               "AR": "AwayTeamRedCards",
                               "B365H": "B365HomeTeam",
                               "B365D": "B365Draw",
                               "B365A": "B365AwayTeam",
                               "MaxH": "MarketMaxHomeTeam",
                               "MaxD": "MarketMaxDraw",
                               "MaxA": "MarketMaxAwayTeam",
                               "AvgH": "MarketAvgHomeTeam",
                               "AvgD": "MarketAvgDraw",
                               "AvgA": "MarketAvgAwayTeam",
                               "B365>2.5": "B365Over2.5Goals",
                               "B365<2.5": "B365Under2.5Goals",
                               "Max>2.5": "MarketMaxOver2.5Goals",
                               "Max<2.5": "MarketMaxUnder2.5Goals",
                               "Avg>2.5": "MarketAvgOver2.5Goals",
                               "Avg<2.5": "MarketAvgUnder2.5Goals"},
                   inplace = True)


In [6]:
# Add season column

website_frame.insert(0, "Season", constants.CURRENT_SEASON_TAG)

In [7]:
# Add MatchWeek value

website_frame.insert(1, "MatchWeek", constants.DEFAULT_MATCHWEEK)

In [8]:
# Add Points columns

conditions = [
     website_frame["FullTimeResult"] == 'H',
     website_frame["FullTimeResult"] == 'D',
     website_frame["FullTimeResult"] == 'A'
]

home_points = [ 3, 1, 0]
away_points = [ 0, 1, 3]

website_frame["HomeTeamPoints"] = np.select(conditions, home_points)
website_frame["AwayTeamPoints"] = np.select(conditions, away_points)

In [9]:
website_frame.shape

(380, 42)

In [10]:
# Stablish a connection to Database data source and fetch all matches stored from current season

try:
    connection = psycopg2.connect(
        host = constants.DB_SERVER,
        port = constants.DB_PORT,
        user = constants.DB_USER,
        password = constants.DB_PASSWORD,
        database = constants.DB_NAME
    )
except psycopg2.Error as e:
    print (f'Can not connect to the postgress database "{constants.DB_NAME}". Make sure database server is running')
    print (e)
else:
    print (f'Connection to database "{constants.DB_NAME}" stablished. Listening at port {constants.DB_PORT}')

season_query = f"SELECT * FROM public.match_history WHERE \"Season\" = '{constants.CURRENT_SEASON_TAG}'"

cursor = connection.cursor()
cursor.execute(season_query)
matches_in_db = cursor.fetchall()

Connection to database "premier_league" stablished. Listening at port 5432


In [11]:
# Copy cursor into a dataframe
postgres_frame = pd.DataFrame(data = matches_in_db, columns = website_frame.columns)

In [12]:
# !! Getting error --> ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects
# This does not work as our dataframes have identical lables, but different indexes as website_frame is always likely to have
# more entries that what we have persisted in DataBase
# new_entries = website_frame.compare(postgres_frame)

In [13]:
# Copy depurated matchweeks from

website_frame.index


RangeIndex(start=0, stop=380, step=1)

In [14]:
postgres_frame.index

RangeIndex(start=0, stop=370, step=1)

In [15]:
# Now let's get only new match entries by contrasting what is comming from website datasource vs what DataBase has


In [16]:
type(matches_in_db)

list

In [17]:
website_frame.head()

Season  MatchWeek        Date   Time     HomeTeam       AwayTeam  \
0  2023-2024          1  11/08/2023  20:00      Burnley       Man City   
1  2023-2024          1  12/08/2023  12:30      Arsenal  Nott'm Forest   
2  2023-2024          1  12/08/2023  15:00  Bournemouth       West Ham   
3  2023-2024          1  12/08/2023  15:00     Brighton          Luton   
4  2023-2024          1  12/08/2023  15:00      Everton         Fulham   

   FullTimeHomeTeamGoals  FullTimeAwayTeamGoals FullTimeResult  \
0                      0                      3              A   
1                      2                      1              H   
2                      1                      1              D   
3                      4                      1              H   
4                      0                      1              A   

   HalfTimeHomeTeamGoals  HalfTimeAwayTeamGoals HalfTimeResult    Referee  \
0                      0                      2              A   C Pawson   
1                      2                      0              H   M Oliver   
2                      0                      0              D   P Bankes   
3                      1                      0              H    D Coote   
4                      0                      0              D  S Attwell   

   HomeTeamShots  AwayTeamShots  HomeTeamShotsOnTarget  AwayTeamShotsOnTarget  \
0              6             17                      1                      8   
1             15              6                      7                      2   
2             14             16                      5                      3   
3             27              9                     12                      3   
4             19              9                      9                      2   

   HomeTeamFouls  AwayTeamFouls  HomeTeamCorners  AwayTeamCorners  \
0             11              8                6                5   
1             12             12                8                3   
2              9             14               10                4   
3             11             12                6                7   
4             12              6               10                4   

   HomeTeamYellowCards  AwayTeamYellowCards  HomeTeamRedCards  \
0                    0                    0                 1   
1                    2                    2                 0   
2                    1                    4                 0   
3                    2                    2                 0   
4                    0                    2                 0   

   AwayTeamRedCards  B365HomeTeam  B365Draw  B365AwayTeam  MarketMaxHomeTeam  \
0                 0          8.00       5.5          1.33               9.50   
1                 0          1.18       7.0         15.00               1.21   
2                 0          2.70       3.4          2.55               2.80   
3                 0          1.33       5.5          9.00               1.36   
4                 0          2.20       3.4          3.30               2.30   

   MarketMaxDraw  MarketMaxAwayTeam  MarketAvgHomeTeam  MarketAvgDraw  \
0           5.68               1.39               9.02           5.35   
1           8.50              17.50               1.18           7.64   
2           3.62               2.75               2.69           3.44   
3           6.00              10.50               1.33           5.52   
4           3.57               3.45               2.24           3.43   

   MarketAvgAwayTeam  B365Over2.5Goals  B365Under2.5Goals  \
0               1.35              1.67               2.20   
1              15.67              1.44               2.75   
2               2.64              1.90               2.00   
3               9.61              1.62               2.30   
4               3.30              2.01               1.89   

   MarketMaxOver2.5Goals  MarketMaxUnder2.5Goals  MarketAvgOver2.5Goals  \
0                   1.71                    2.